In [48]:
import pandas
import seaborn
import numpy

import sklearn.preprocessing
import sklearn.feature_selection

%matplotlib inline

# Instructions

- Read both training and test data from CSV files
- Then, identify which columns are numerical and which columns are categorical
    - Note that some of numerical feature are actually categorical feature
- Assess whether the data is balance or imbalance

In [49]:
data_train = pandas.read_csv('../data/credit.train.csv').set_index('id')
data_train

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,label
id,,,,,,,,,,,,,,,,,,,,,
253,no checking,24.0,delayed previously,furniture/equipment,4151.0,100<=X<500,1<=X<4,2,male single,none,...,life insurance,35,none,own,2.0,skilled,1,none,yes,good
667,no checking,48.0,all paid,business,3609.0,<100,1<=X<4,1,female div/dep/mar,none,...,real estate,27,stores,own,1.0,skilled,1,none,yes,good
85,no checking,12.0,critical/other existing credit,business,1412.0,<100,1<=X<4,4,female div/dep/mar,guarantor,...,real estate,29,none,own,2.0,high qualif/self emp/mgmt,1,yes,yes,good
969,<0,11.0,critical/other existing credit,new car,3939.0,<100,1<=X<4,1,male single,none,...,real estate,40,none,own,2.0,unskilled resident,2,none,yes,good
75,<0,12.0,critical/other existing credit,used car,1526.0,<100,>=7,4,male single,none,...,no known property,66,none,for free,2.0,high qualif/self emp/mgmt,1,none,yes,good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,<0,12.0,no credits/all paid,new car,1082.0,<100,1<=X<4,4,male single,none,...,car,48,bank,own,2.0,skilled,1,none,yes,bad
192,0<=X<200,27.0,existing paid,business,3915.0,<100,1<=X<4,4,male single,none,...,car,36,none,own,1.0,skilled,2,yes,yes,bad
629,no checking,9.0,existing paid,education,3832.0,no known savings,>=7,1,male single,none,...,real estate,64,none,own,1.0,unskilled resident,1,none,yes,good


In [50]:
data_test = pandas.read_csv('../data/credit.test.csv').set_index('id')
data_test

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,label
id,,,,,,,,,,,,,,,,,,,,,
993,<0,36.0,existing paid,furniture/equipment,3959.0,<100,unemployed,4,male single,none,...,life insurance,30,none,own,1.0,high qualif/self emp/mgmt,1,yes,yes,good
859,no checking,9.0,existing paid,new car,3577.0,100<=X<500,1<=X<4,1,male single,guarantor,...,real estate,26,none,rent,1.0,skilled,2,none,no,good
298,no checking,18.0,existing paid,furniture/equipment,2515.0,<100,1<=X<4,3,male single,none,...,real estate,43,none,own,1.0,skilled,1,yes,yes,good
553,0<=X<200,12.0,critical/other existing credit,new car,1995.0,100<=X<500,<1,4,male single,none,...,car,27,none,own,1.0,skilled,1,none,yes,good
672,no checking,60.0,existing paid,new car,10366.0,<100,>=7,2,male single,none,...,life insurance,42,none,own,1.0,high qualif/self emp/mgmt,1,yes,yes,good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,0<=X<200,12.0,existing paid,furniture/equipment,3017.0,<100,<1,3,female div/dep/mar,none,...,real estate,34,none,rent,1.0,high qualif/self emp/mgmt,1,none,yes,good
356,no checking,12.0,critical/other existing credit,radio/tv,2331.0,no known savings,>=7,1,male single,co applicant,...,real estate,49,none,own,1.0,skilled,1,yes,yes,good
2,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,2,male single,none,...,real estate,49,none,own,1.0,unskilled resident,2,none,yes,good


In [51]:
data_train.dtypes

checking_status            object
duration                  float64
credit_history             object
purpose                    object
credit_amount             float64
savings_status             object
employment                 object
installment_commitment      int64
personal_status            object
other_parties              object
residence_since           float64
property_magnitude         object
age                         int64
other_payment_plans        object
housing                    object
existing_credits          float64
job                        object
num_dependents              int64
own_telephone              object
foreign_worker             object
label                      object
dtype: object

In [52]:
categorical_features = [
    'checking_status', 
    'credit_history', 
    'purpose', 
    'savings_status',
    'employment', 
    'personal_status', 
    'other_parties', 
    'property_magnitude',
    'other_payment_plans', 
    'housing', 
    'job', 
    'own_telephone',
    'foreign_worker',
    'installment_commitment', 
    'residence_since', 
    'existing_credits', 
    'num_dependents'
]

for feature in categorical_features:
    possible_values = data_train[feature].unique()
    print(f"{feature:20s} {possible_values.size:2d} {possible_values}")

checking_status       5 ['no checking' '<0' '0<=X<200' '>=200' nan]
credit_history        5 ['delayed previously' 'all paid' 'critical/other existing credit'
 'existing paid' 'no credits/all paid']
purpose              11 ['furniture/equipment' 'business' 'new car' 'used car' 'radio/tv'
 'domestic appliance' 'repairs' 'other' 'education' 'retraining' nan]
savings_status        5 ['100<=X<500' '<100' '500<=X<1000' '>=1000' 'no known savings']
employment            5 ['1<=X<4' '>=7' '<1' '4<=X<7' 'unemployed']
personal_status       5 ['male single' 'female div/dep/mar' 'male mar/wid' 'male div/sep' nan]
other_parties         3 ['none' 'guarantor' 'co applicant']
property_magnitude    5 ['life insurance' 'real estate' 'no known property' 'car' nan]
other_payment_plans   3 ['none' 'stores' 'bank']
housing               4 ['own' 'for free' 'rent' nan]
job                   5 ['skilled' 'high qualif/self emp/mgmt' 'unskilled resident'
 'unemp/unskilled non res' nan]
own_telephone         3 [

In [53]:
data_train['label'].value_counts()

label
good    522
bad     226
Name: count, dtype: int64

In [54]:
data_test['label'].value_counts()

label
good    176
bad      74
Name: count, dtype: int64

# Instructions
- Investigate whether the dataset contains any missing data

In [55]:
data_train.isna().sum(axis='index')

checking_status           2
duration                  1
credit_history            0
purpose                   1
credit_amount             1
savings_status            0
employment                0
installment_commitment    0
personal_status           2
other_parties             0
residence_since           1
property_magnitude        1
age                       0
other_payment_plans       0
housing                   1
existing_credits          2
job                       1
num_dependents            0
own_telephone             1
foreign_worker            1
label                     2
dtype: int64

In [56]:
data_test.isna().sum(axis='index')

checking_status           1
duration                  0
credit_history            0
purpose                   1
credit_amount             0
savings_status            0
employment                0
installment_commitment    0
personal_status           0
other_parties             0
residence_since           0
property_magnitude        0
age                       0
other_payment_plans       0
housing                   0
existing_credits          1
job                       0
num_dependents            0
own_telephone             0
foreign_worker            0
label                     0
dtype: int64

# Instructions
- Remove missing values
- Create 4 `pandas.DataFrame`s
    - `pandas.DataFrame` of ordinal features encoded using `sklearn.preprocessing.OrdinalEncoder`
    - `pandas.DataFrame` of only nominal features encoded using `sklearn.preprocessing.OneHotEncoder`
    - `pandas.DataFrame` of only numerical features. Also, apply `sklearn.preprocessing.StandardScaler`.
    - `pandas.DataFrame` of labels encoded using `sklearn.preprocessing.OrdinalEncoder` 
       - Map {`good`, `bad`} to {0, 1} respectively
- Note that these **filling parameters** should be applied to the test data as well
- Then, combine the 3 `pandas.DataFrame` to obtain the fully processed training and test data
  - Hint: use `.join`


## For missing values
- Drop all missing values for training and testing data
    - Hint: use `.dropna`
- Show that the missing values were properly removed

In [57]:
data_train = data_train.dropna()

In [58]:
data_train.isna().sum(axis='index')

checking_status           0
duration                  0
credit_history            0
purpose                   0
credit_amount             0
savings_status            0
employment                0
installment_commitment    0
personal_status           0
other_parties             0
residence_since           0
property_magnitude        0
age                       0
other_payment_plans       0
housing                   0
existing_credits          0
job                       0
num_dependents            0
own_telephone             0
foreign_worker            0
label                     0
dtype: int64

In [59]:
data_test = data_test.dropna()

In [60]:
data_test.isna().sum(axis='index')

checking_status           0
duration                  0
credit_history            0
purpose                   0
credit_amount             0
savings_status            0
employment                0
installment_commitment    0
personal_status           0
other_parties             0
residence_since           0
property_magnitude        0
age                       0
other_payment_plans       0
housing                   0
existing_credits          0
job                       0
num_dependents            0
own_telephone             0
foreign_worker            0
label                     0
dtype: int64

## For ordinal features
- Define a subset of ordinal features
- Define a list of categorical maps which sort the feature values in appropriate orders
- Use `sklearn.preprocessing.OrdinalEncoder` to create a categorical encoder, then fit using the training data
- Create a new `pandas.DataFrame` of only ordinally encoded features for both training and test data

In [61]:
ordinal_features = [
    'employment', 
    'own_telephone',
    'foreign_worker',
    'installment_commitment',
    'residence_since',
    'existing_credits',
    'num_dependents',
]

In [62]:
ordinal_mapping = [
    ['unemployed', '<1', '1<=X<4', '4<=X<7', '>=7'],
    ['none', 'yes'],
    ['no', 'yes'],
    [1, 2, 3, 4],
    [1.0, 2.0, 3.0, 4.0],
    [1.0, 2.0, 3.0, 4.0],
    [1, 2],
]

In [63]:
ordinal_encoder = sklearn.preprocessing.OrdinalEncoder(categories=ordinal_mapping)
ordinal_encoder

OrdinalEncoder(categories=[['unemployed', '<1', '1<=X<4', '4<=X<7', '>=7'],
                           ['none', 'yes'], ['no', 'yes'], [1, 2, 3, 4],
                           [1.0, 2.0, 3.0, 4.0], [1.0, 2.0, 3.0, 4.0], [1, 2]])

In [64]:
ordinal_encoder.fit(
    X = data_train[ordinal_features]
)

OrdinalEncoder(categories=[['unemployed', '<1', '1<=X<4', '4<=X<7', '>=7'],
                           ['none', 'yes'], ['no', 'yes'], [1, 2, 3, 4],
                           [1.0, 2.0, 3.0, 4.0], [1.0, 2.0, 3.0, 4.0], [1, 2]])

In [65]:
data_train_ordinal_features = pandas.DataFrame(
    data    = ordinal_encoder.transform(data_train[ordinal_features]),
    columns = ordinal_encoder.get_feature_names_out(),
    index   = data_train.index
)
data_train_ordinal_features

,employment,own_telephone,foreign_worker,installment_commitment,residence_since,existing_credits,num_dependents
id,,,,,,,
253,2.0,0.0,1.0,1.0,2.0,1.0,0.0
667,2.0,0.0,1.0,0.0,0.0,0.0,0.0
85,2.0,1.0,1.0,3.0,1.0,1.0,0.0
969,2.0,0.0,1.0,0.0,1.0,1.0,1.0
75,4.0,0.0,1.0,3.0,3.0,1.0,0.0
...,...,...,...,...,...,...,...
835,2.0,0.0,1.0,3.0,3.0,1.0,0.0
192,2.0,1.0,1.0,3.0,1.0,0.0,1.0
629,4.0,0.0,1.0,0.0,3.0,0.0,0.0


In [66]:
data_test_ordinal_features = pandas.DataFrame(
    data    = ordinal_encoder.transform(data_test[ordinal_features]),
    columns = ordinal_encoder.get_feature_names_out(),
    index   = data_test.index
)
data_test_ordinal_features

,employment,own_telephone,foreign_worker,installment_commitment,residence_since,existing_credits,num_dependents
id,,,,,,,
993,0.0,1.0,1.0,3.0,2.0,0.0,0.0
859,2.0,0.0,0.0,0.0,1.0,0.0,1.0
298,2.0,1.0,1.0,2.0,3.0,0.0,0.0
553,1.0,0.0,1.0,3.0,0.0,0.0,0.0
672,4.0,1.0,1.0,1.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...
462,1.0,0.0,1.0,2.0,0.0,0.0,0.0
356,4.0,1.0,1.0,0.0,3.0,0.0,0.0
2,3.0,0.0,1.0,1.0,2.0,0.0,1.0


## For nominal features
- Define a subset of nominal features
- Define a list of nominal maps which sort the feature values in appropriate orders
- Use `sklearn.preprocessing.OneHotEncoder` to create an nominal encoder, then fit using the training data
- Create a new `pandas.DataFrame` of only nominally encoded features for both training and test data

In [67]:
nominal_features = [
    'job', 
    'checking_status', 
    'credit_history', 
    'purpose', 
    'savings_status',
    'personal_status', 
    'other_parties', 
    'property_magnitude',
    'other_payment_plans', 
    'housing', 
]

In [68]:
nominal_mapping = [
    [ 'unemp/unskilled non res', 'unskilled resident', 'skilled', 'high qualif/self emp/mgmt'],
    ['no checking', '<0', '0<=X<200', '>=200'],
    ['delayed previously', 'all paid', 'critical/other existing credit', 'existing paid', 'no credits/all paid'],
    ['furniture/equipment', 'business', 'new car', 'used car', 'radio/tv', 'domestic appliance', 'repairs', 'education', 'retraining', 'other'],
    ['no known savings', '<100', '100<=X<500', '500<=X<1000', '>=1000'],
    ['male single', 'female div/dep/mar', 'male mar/wid', 'male div/sep'],
    ['none', 'guarantor', 'co applicant'],
    ['no known property', 'life insurance', 'car', 'real estate'],
    ['none', 'stores', 'bank'],
    ['for free', 'rent', 'own']
]

In [69]:
onehot_encoder = sklearn.preprocessing.OneHotEncoder(categories=nominal_mapping, sparse_output=False)
onehot_encoder

OneHotEncoder(categories=[['unemp/unskilled non res', 'unskilled resident',
                           'skilled', 'high qualif/self emp/mgmt'],
                          ['no checking', '<0', '0<=X<200', '>=200'],
                          ['delayed previously', 'all paid',
                           'critical/other existing credit', 'existing paid',
                           'no credits/all paid'],
                          ['furniture/equipment', 'business', 'new car',
                           'used car', 'radio/tv', 'domestic appliance',
                           'repairs', 'education', 'retraining', 'other'],
                          ['no known savings', '<100', '100<=X<500',
                           '500<=X<1000', '>=1000'],
                          ['male single', 'female div/dep/mar', 'male mar/wid',
                           'male div/sep'],
                          ['none', 'guarantor', 'co applicant'],
                          ['no known property', 'life insurance', 'car',
                           'real estate'],
                          ['none', 'stores', 'bank'],
                          ['for free', 'rent', 'own']],
              sparse_output=False)

In [70]:
onehot_encoder.fit(
    X=data_train[nominal_features]
)

OneHotEncoder(categories=[['unemp/unskilled non res', 'unskilled resident',
                           'skilled', 'high qualif/self emp/mgmt'],
                          ['no checking', '<0', '0<=X<200', '>=200'],
                          ['delayed previously', 'all paid',
                           'critical/other existing credit', 'existing paid',
                           'no credits/all paid'],
                          ['furniture/equipment', 'business', 'new car',
                           'used car', 'radio/tv', 'domestic appliance',
                           'repairs', 'education', 'retraining', 'other'],
                          ['no known savings', '<100', '100<=X<500',
                           '500<=X<1000', '>=1000'],
                          ['male single', 'female div/dep/mar', 'male mar/wid',
                           'male div/sep'],
                          ['none', 'guarantor', 'co applicant'],
                          ['no known property', 'life insurance', 'car',
                           'real estate'],
                          ['none', 'stores', 'bank'],
                          ['for free', 'rent', 'own']],
              sparse_output=False)

In [71]:
onehot_encoder.get_feature_names_out()

array(['job_unemp/unskilled non res', 'job_unskilled resident',
       'job_skilled', 'job_high qualif/self emp/mgmt',
       'checking_status_no checking', 'checking_status_<0',
       'checking_status_0<=X<200', 'checking_status_>=200',
       'credit_history_delayed previously', 'credit_history_all paid',
       'credit_history_critical/other existing credit',
       'credit_history_existing paid',
       'credit_history_no credits/all paid',
       'purpose_furniture/equipment', 'purpose_business',
       'purpose_new car', 'purpose_used car', 'purpose_radio/tv',
       'purpose_domestic appliance', 'purpose_repairs',
       'purpose_education', 'purpose_retraining', 'purpose_other',
       'savings_status_no known savings', 'savings_status_<100',
       'savings_status_100<=X<500', 'savings_status_500<=X<1000',
       'savings_status_>=1000', 'personal_status_male single',
       'personal_status_female div/dep/mar',
       'personal_status_male mar/wid', 'personal_status_male div

In [72]:
data_train_nominal_features = pandas.DataFrame(
    data    = onehot_encoder.transform(data_train[nominal_features]),
    columns = onehot_encoder.get_feature_names_out(),
    index   = data_train.index
)
data_train_nominal_features

,job_unemp/unskilled non res,job_unskilled resident,job_skilled,job_high qualif/self emp/mgmt,checking_status_no checking,checking_status_<0,checking_status_0<=X<200,checking_status_>=200,credit_history_delayed previously,credit_history_all paid,...,property_magnitude_no known property,property_magnitude_life insurance,property_magnitude_car,property_magnitude_real estate,other_payment_plans_none,other_payment_plans_stores,other_payment_plans_bank,housing_for free,housing_rent,housing_own
id,,,,,,,,,,,,,,,,,,,,,
253,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
667,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
85,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
969,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
75,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
192,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
629,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


In [73]:
data_test_nominal_features = pandas.DataFrame(
    data    = onehot_encoder.transform(data_test[nominal_features]),
    columns = onehot_encoder.get_feature_names_out(),
    index   = data_test.index
)
data_test_nominal_features

,job_unemp/unskilled non res,job_unskilled resident,job_skilled,job_high qualif/self emp/mgmt,checking_status_no checking,checking_status_<0,checking_status_0<=X<200,checking_status_>=200,credit_history_delayed previously,credit_history_all paid,...,property_magnitude_no known property,property_magnitude_life insurance,property_magnitude_car,property_magnitude_real estate,other_payment_plans_none,other_payment_plans_stores,other_payment_plans_bank,housing_for free,housing_rent,housing_own
id,,,,,,,,,,,,,,,,,,,,,
993,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
859,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
298,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
553,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
672,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
356,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


## For numerical features
- Define and select a subset of numerical features for training and test data
- Use `sklearn.preprocessing.StandardScaler` to create a standard scaler, then fit using the training data
- Create a new `pandas.DataFrame` of only numerically encoded features for both training and test data

In [74]:
numerical_features = [
    'duration',
    'credit_amount',
    'age',
]

In [75]:
data_train_numerical_features = data_train[numerical_features]
data_train_numerical_features

,duration,credit_amount,age
id,,,
253,24.0,4151.0,35
667,48.0,3609.0,27
85,12.0,1412.0,29
969,11.0,3939.0,40
75,12.0,1526.0,66
...,...,...,...
835,12.0,1082.0,48
192,27.0,3915.0,36
629,9.0,3832.0,64


In [76]:
data_test_numerical_features = data_test[numerical_features]
data_test_numerical_features

,duration,credit_amount,age
id,,,
993,36.0,3959.0,30
859,9.0,3577.0,26
298,18.0,2515.0,43
553,12.0,1995.0,27
672,60.0,10366.0,42
...,...,...,...
462,12.0,3017.0,34
356,12.0,2331.0,49
2,12.0,2096.0,49


In [77]:
standard_scaler = sklearn.preprocessing.StandardScaler()
standard_scaler

StandardScaler()

In [78]:
standard_scaler.fit(data_train_numerical_features)

StandardScaler()

In [79]:
data_train_numerical_features = pandas.DataFrame(
    data    = standard_scaler.transform(data_train_numerical_features),
    columns = standard_scaler.get_feature_names_out(),
    index   = data_train_numerical_features.index
)
data_train_numerical_features

,duration,credit_amount,age
id,,,
253,0.284625,0.315530,-0.079689
667,2.308525,0.126008,-0.763872
85,-0.727325,-0.642219,-0.592826
969,-0.811654,0.241399,0.347925
75,-0.727325,-0.602357,2.571519
...,...,...,...
835,-0.727325,-0.757610,1.032108
192,0.537613,0.233007,0.005834
629,-0.980312,0.203985,2.400474


In [80]:
data_test_numerical_features = pandas.DataFrame(
    data    = standard_scaler.transform(data_test_numerical_features),
    columns = standard_scaler.get_feature_names_out(),
    index   = data_test_numerical_features.index
)
data_test_numerical_features

,duration,credit_amount,age
id,,,
993,1.296575,0.248393,-0.507303
859,-0.980312,0.114818,-0.849395
298,-0.221350,-0.256532,0.604494
553,-0.727325,-0.438361,-0.763872
672,3.320475,2.488734,0.518971
...,...,...,...
462,-0.727325,-0.080997,-0.165212
356,-0.727325,-0.320871,1.117631
2,-0.727325,-0.403044,1.117631


## For labels
- Define a subset of ordinal features
- Use `sklearn.preprocessing.OrdinalEncoder` to create an ordinal encoder, then fit using the training data
  - Map {`good`, `bad`} to {0, 1} respectively
- Create a new `pandas.DataFrame` of ordinally encoded labels for both training and test data

In [81]:
label_encoder = sklearn.preprocessing.OrdinalEncoder(categories=[['good','bad']])
label_encoder

OrdinalEncoder(categories=[['good', 'bad']])

In [82]:
label_encoder.fit(data_train[['label']])

OrdinalEncoder(categories=[['good', 'bad']])

In [83]:
data_train_label = pandas.DataFrame(
    data    = label_encoder.transform(data_train[['label']]),
    columns = ['label'],
    index   = data_train.index
)
data_train_label

,label
id,
253,0.0
667,0.0
85,0.0
969,0.0
75,0.0
...,...
835,1.0
192,1.0
629,0.0


In [84]:
data_test_label = pandas.DataFrame(
    data    = label_encoder.transform(data_test[['label']]),
    columns = ['label'],
    index   = data_test.index
)
data_test_label

,label
id,
993,0.0
859,0.0
298,0.0
553,0.0
672,0.0
...,...
462,0.0
356,0.0
2,0.0


## Reconstruction of data sets
- Combine the 4 `pandas.DataFrame` to obtain the fully processed training and test data
  - Hint: use `.join`

In [85]:
data_train = (
    data_train_numerical_features
        .join(data_train_ordinal_features)
        .join(data_train_nominal_features)
        .join(data_train_label)
)
data_train

,duration,credit_amount,age,employment,own_telephone,foreign_worker,installment_commitment,residence_since,existing_credits,num_dependents,...,property_magnitude_life insurance,property_magnitude_car,property_magnitude_real estate,other_payment_plans_none,other_payment_plans_stores,other_payment_plans_bank,housing_for free,housing_rent,housing_own,label
id,,,,,,,,,,,,,,,,,,,,,
253,0.284625,0.315530,-0.079689,2.0,0.0,1.0,1.0,2.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
667,2.308525,0.126008,-0.763872,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
85,-0.727325,-0.642219,-0.592826,2.0,1.0,1.0,3.0,1.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
969,-0.811654,0.241399,0.347925,2.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
75,-0.727325,-0.602357,2.571519,4.0,0.0,1.0,3.0,3.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,-0.727325,-0.757610,1.032108,2.0,0.0,1.0,3.0,3.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
192,0.537613,0.233007,0.005834,2.0,1.0,1.0,3.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
629,-0.980312,0.203985,2.400474,4.0,0.0,1.0,0.0,3.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [86]:
data_test = (
    data_test_numerical_features
        .join(data_test_ordinal_features)
        .join(data_test_nominal_features)
        .join(data_test_label)
)
data_test

,duration,credit_amount,age,employment,own_telephone,foreign_worker,installment_commitment,residence_since,existing_credits,num_dependents,...,property_magnitude_life insurance,property_magnitude_car,property_magnitude_real estate,other_payment_plans_none,other_payment_plans_stores,other_payment_plans_bank,housing_for free,housing_rent,housing_own,label
id,,,,,,,,,,,,,,,,,,,,,
993,1.296575,0.248393,-0.507303,0.0,1.0,1.0,3.0,2.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
859,-0.980312,0.114818,-0.849395,2.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
298,-0.221350,-0.256532,0.604494,2.0,1.0,1.0,2.0,3.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
553,-0.727325,-0.438361,-0.763872,1.0,0.0,1.0,3.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
672,3.320475,2.488734,0.518971,4.0,1.0,1.0,1.0,3.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,-0.727325,-0.080997,-0.165212,1.0,0.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
356,-0.727325,-0.320871,1.117631,4.0,1.0,1.0,0.0,3.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.727325,-0.403044,1.117631,3.0,0.0,1.0,1.0,2.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


# Instruction

- Define a `sklearn.neighbors.KNeighborsClassifier` model with the following configurations to be used as an estimator
```
        n_neighbors = 5,
        weights     = 'uniform', 
        p           = 2, 
```
    
- Use `sklearn.feature_selection.SequentialFeatureSelector` with the following configurations for automatic feature selection

```
        estimator            = model,
        n_features_to_select = 15,
        direction            = 'backward'

```

- Now, fit the feature selector to the training data


KNeighborsClassifier()

SequentialFeatureSelector(direction='backward',
                          estimator=KNeighborsClassifier(),
                          n_features_to_select=15)

SequentialFeatureSelector(direction='backward',
                          estimator=KNeighborsClassifier(),
                          n_features_to_select=15)

# Instructions
- Use `.get_feature_names_out()` to get the optimal subset of features from the feature selector
- Create `pandas.DataFrame`s that contain only those selected features for training data and test data respectively 

['duration',
 'credit_amount',
 'age',
 'employment',
 'own_telephone',
 'residence_since',
 'num_dependents',
 'checking_status_no checking',
 'checking_status_<0',
 'credit_history_critical/other existing credit',
 'purpose_furniture/equipment',
 'purpose_radio/tv',
 'property_magnitude_no known property',
 'property_magnitude_life insurance',
 'housing_rent']

# Instructions
- Write the train data set to `./data/features.train.csv`
- Write the test data set to `./data/features.test.csv`